# Исследование надежности заемщиков
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from scipy import stats
from scipy.stats import chi2_contingency
import seaborn as sns

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В файле 12 колонок, 2 колонки типа float, 5 колонок типа int и 5 колонок типа object.

## Шаг 2. Предобработка данных

Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

`total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Обработаем аномальные значения в стообце `days_employed` и выделим для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.     

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости `income_type`.

In [9]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Посмотрим на уникальные значения столбца `children`.**

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)] #избавляемся от аномальных значений

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [13]:
data.isna().sum() # убедимся, что пропусков в данных больше нет

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв

In [14]:
data['education'] = data['education'].str.lower() # 

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates() # удаляем дубликаты

### Категоризация данных

**На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Сформируем новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data_pivot = pd.crosstab(data.children,data.debt)
print(data_pivot)


#добавлю таблицу сопряженности.
data_pivot_1 = data.pivot_table(index='children',values='debt',aggfunc=['count','sum'])
data_pivot_1['ratio%'] = data_pivot_1['sum', 'debt'] / data_pivot_1['count', 'debt'] * 100
print(data_pivot_1.head())
print(chi2_contingency(data_pivot))

debt          0     1
children             
0         13028  1063
1          4364   444
2          1858   194
3           303    27
4            37     4
5             9     0
          count   sum    ratio%
           debt  debt          
children                       
0         14091  1063  7.543822
1          4808   444  9.234609
2          2052   194  9.454191
3           330    27  8.181818
4            41     4  9.756098
Chi2ContingencyResult(statistic=20.117395586863267, pvalue=0.0011878936268035906, dof=5, expected_freq=array([[1.29468618e+04, 1.14413820e+03],
       [4.41760780e+03, 3.90392199e+02],
       [1.88538503e+03, 1.66614974e+02],
       [3.03205194e+02, 2.67948057e+01],
       [3.76709484e+01, 3.32905162e+00],
       [8.26923257e+00, 7.30767428e-01]]))


Тест хи-квадрат позволяет находит связи между двумя номинальными переменными на основе таблицы сопряженности 
и сравнениях ожидаемых и наблюдаемых частот.
Гипотеза, которую мы проверяем: Количество детей и возврат кредита в срок подчиняется нормальному закону распределения.
Х2 = 20.117, df = 5, p-value < 0.05. Можем отклонить нулевую гипотезу об отсутсвии связи между количеством детей и возвратом кредита в срок.
По остаткам: количество людей без детей и без долгов больше ожидания, а с 1 ребенком и без долгов - самую малость меньше ожидания. 

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data_pivot2_2 = data.pivot_table(index='family_status',values='debt',aggfunc=['count','sum'])
data_pivot2_2['ratio%'] = data_pivot2_2['sum', 'debt'] / data_pivot2_2['count', 'debt'] * 100
print(data_pivot2_2.head())

                       count  sum    ratio%
                        debt debt          
family_status                              
Не женат / не замужем   2796  273  9.763948
в разводе               1189   84  7.064760
вдовец / вдова           951   63  6.624606
гражданский брак        4134  385  9.313014
женат / замужем        12261  927  7.560558


Незамужные и неженатые(в том числе и проживающие в гражданском браке) реже остальных отдают кредиты в срок.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data_pivot3 = data.pivot_table(index='total_income_category',values='debt',aggfunc=['count','sum'])
data_pivot3['ratio%'] = data_pivot3['sum', 'debt'] / data_pivot3['count', 'debt'] * 100
data_pivot3['%'] = data_pivot3['count','debt'] / data_pivot3['count','debt'].sum() * 100
print(data_pivot3)

                       count   sum    ratio%          %
                        debt  debt                     
total_income_category                                  
A                         25     2  8.000000   0.117200
B                       5014   354  7.060231  23.505696
C                      15921  1353  8.498210  74.637851
D                        349    21  6.017192   1.636116
E                         22     2  9.090909   0.103136


In [25]:
stats.ttest_ind(data[data['debt'] == 0]['total_income'],data[data['debt'] == 1]['total_income'])

Ttest_indResult(statistic=1.850694711638729, pvalue=0.06422730260815598)

Результаты Т-теста: p-value > 0.05, не можем отклонить нулевую гипотезу(отутствие свзяи между уровнем дохода и возвратом кредита в срок)

Конверсия: если рассматривать категории B и С (состовляют 98% всей выборки): респонденты из категории B чаще возвращают кредит в срок.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [26]:
data_pivot4 = data.pivot_table(index='purpose_category',values='debt',aggfunc=['count','sum'])
data_pivot4['ratio%'] = data_pivot4['sum', 'debt'] / data_pivot4['count', 'debt'] * 100
print(data_pivot4)

                          count  sum    ratio%
                           debt debt          
purpose_category                              
операции с автомобилем     4279  400  9.347978
операции с недвижимостью  10751  780  7.255139
получение образования      3988  369  9.252758
проведение свадьбы         2313  183  7.911803


Кредиты на образование и операции с авто возвращают реже, чем в остальных категориях.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

Пропуски в строке ежемесечного дохода, если данные заполнял сам человек, скорее всего обусловлены нежеланием человека распространятся о своих доходах или сложностью в их подсчете. Избежать таких пропусков можно используя закрытый-альтернативный вид формулировки вопроса.

Пропуски в общем трудовом стаже, скорее всего, тоже связаны с трудностью подсчета. 

### Шаг 4: общий вывод.

Цель исследования надежности займщиков: формулировка и проверка гипотез о наличии зависимости между переменным.

В результате исследования были подтверждены подтверждены гипотезы:

Существует зависимость между количеством детей и возвратом кредита в срок. Отсутсвие ребенка положительно сказывается на вероятности возвращения кредита без просрочки. 
Существует зависимость между семейным положением и возвратом кредита в срок. Официально незамужные люди возвращают кредиты реже других. Вдовы и разведенные самые надежные клиенты.
Существует зависимость между уровнем дохода и возвратом кредита в срок. Чем больше доход, тем чаще возвращают кредит во время. 
Цели кредита также влияют на вероятность возвращения его(кредита) в срок. Кредит на операцию с недвижимостью и свадьбу обладает наибольшей конверсией. 